### 8.1. 개인 대출 수락. UniversalBank.csv파일에는 유니버설 은행의 5000명 고객에 대한 데이터가 있다. 이 데이터에는 고객의 인구통계학적 정보(나이, 수입 등), 고객과 은행 간 관계(저당, 안심 계좌 등), 지난 개인 대출 캠페인에 대한 고객의 응답(개인 대출) 정보가 담겨 있다. 고객 5000명 중 480명(=9.6%)만 이전 캠페인에서 개인 대출을 수락했다. 이 문제에서는 2개의 예측 변수, 즉 온라인(고객이 온라인 뱅킹 서비스의 적극적인 사용자인지 여부)과 신용카드(이후 CC로 표시됨: 고객이 은행에서 발행한 신용카드르르 소유하고 있는지 여부)에 초점을 맞춘다. 출력 변수는 개인 대출(이후 Loan으로 표시됨)이다.

a. 학습 데이터셋에 대해서 Online을 열 변수로, CC를 첫 번째 행 변수로, Loan을 두 번째 행 변수로 하는 피벗 테이블을 생성하시오. 셀 내부의 값들은 개수로 표시한다. pandas의 데이터 프레임 함수 melt()와 pivot()을 사용한다.

In [ ]:
!pip install dmba

import pandas as pd
import dmba

# Load the dataset
bank_data = dmba.load_data('UniversalBank.csv')

# Display the first few rows to understand the structure of the data
bank_data.head()


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [ ]:
# Create a pivot table with 'Online' as column, 'CreditCard' (CC) as first row variable,
# and 'Personal Loan' (Loan) as the second row variable. Count the occurrences in each cell.

# Selecting relevant columns
data_subset = bank_data[['Online', 'CreditCard', 'Personal Loan']]

# Melting the data for pivot operation
melted_data = data_subset.melt(id_vars=['Online'], var_name='CC_Loan', value_name='Value')

# Creating the pivot table
pivot_table = melted_data.pivot_table(index='CC_Loan', columns='Online', values='Value', aggfunc='count')

# Display the pivot table
pivot_table


Online,0,1
CC_Loan,,
CreditCard,2016,2984
Personal Loan,2016,2984


b. 은행 신용카드를 소유하고 온라인 뱅킹 서비스를 적극적으로 사용하는 고객을 분류하는 문제를 생각해 보자. 피벗 테이블에서 보면 이 고객이 대출 제안을 받아들일 확률은 얼마인가?[이는 은행 신용카드를 갖고 (CC=1)온라인 뱅킹 서비스의 적극적인 사용자(Online=1) 인 조건에서 대출을 수락할 확률(Loan=1)할 확률이다.]

9.3%

In [ ]:
# Filter data for customers who have a CreditCard (CC=1) and are active users of Online banking (Online=1)
filtered_data = bank_data[(bank_data['CreditCard'] == 1) & (bank_data['Online'] == 1)]

# Calculate the probability of accepting a loan (Loan=1) under these conditions
loan_acceptance_rate = filtered_data['Personal Loan'].mean()

# Display the calculated probability
loan_acceptance_rate


0.09297052154195011

c. 이 학습 데이터셋으로부터 2개의 독자적인 피벗 테이블을 작성하시오. 하나는 Online(열)의 함수로 Loan(행)을 가질 것이고, 다른 하나는 CC의 함수로 Loan(행)을 가질 것이다.

In [ ]:
# Creating the first pivot table with 'Online' as columns and 'Loan' as rows
pivot_table_online = bank_data.pivot_table(index='Personal Loan', columns='Online', aggfunc='size', fill_value=0)

# Creating the second pivot table with 'CreditCard' as columns and 'Loan' as rows
pivot_table_cc = bank_data.pivot_table(index='Personal Loan', columns='CreditCard', aggfunc='size', fill_value=0)

# Display both pivot tables
pivot_table_online, pivot_table_cc


(Online            0     1
 Personal Loan            
 0              1827  2693
 1               189   291,
 CreditCard        0     1
 Personal Loan            
 0              3193  1327
 1               337   143)

d. 다음 값을 계산하시오. [$P(A|B)$는 "B가 주어졌을 때 A의 확률"을 의미한다.]
  1. P(CC=1|Loan=1)(대출 수락자들 중에서 신용카드 소유자의 비율): 29.8%
  2. P(Online=1|Loan=1): 60.6%
  3. P(Loan=1)(대출 수락자들의 비율): 9.6%
  4. P(CC=1|Loan=0): 29.4%
  5. P(Online=1|Loan=0): 59.6%
  6. P(Loan=0): 90.4%

In [ ]:
# Calculating the required conditional probabilities

# 1. P(CC=1 | Loan=1) - Proportion of loan accepters who own a credit card
p_cc_given_loan = bank_data[(bank_data['CreditCard'] == 1) & (bank_data['Personal Loan'] == 1)].shape[0] / \
                  bank_data[bank_data['Personal Loan'] == 1].shape[0]

# 2. P(Online=1 | Loan=1) - Proportion of loan accepters who are active online banking users
p_online_given_loan = bank_data[(bank_data['Online'] == 1) & (bank_data['Personal Loan'] == 1)].shape[0] / \
                      bank_data[bank_data['Personal Loan'] == 1].shape[0]

# 3. P(Loan=1) - Proportion of customers who accepted the loan
p_loan_accepted = bank_data[bank_data['Personal Loan'] == 1].shape[0] / bank_data.shape[0]

# 4. P(CC=1 | Loan=0) - Proportion of non-loan accepters who own a credit card
p_cc_given_not_loan = bank_data[(bank_data['CreditCard'] == 1) & (bank_data['Personal Loan'] == 0)].shape[0] / \
                      bank_data[bank_data['Personal Loan'] == 0].shape[0]

# 5. P(Online=1 | Loan=0) - Proportion of non-loan accepters who are active online banking users
p_online_given_not_loan = bank_data[(bank_data['Online'] == 1) & (bank_data['Personal Loan'] == 0)].shape[0] / \
                          bank_data[bank_data['Personal Loan'] == 0].shape[0]

# 6. P(Loan=0) - Proportion of customers who did not accept the loan
p_loan_not_accepted = bank_data[bank_data['Personal Loan'] == 0].shape[0] / bank_data.shape[0]

# Displaying the results
p_cc_given_loan, p_online_given_loan, p_loan_accepted, p_cc_given_not_loan, p_online_given_not_loan, p_loan_not_accepted


(0.29791666666666666,
 0.60625,
 0.096,
 0.29358407079646015,
 0.5957964601769912,
 0.904)

e. 위에서 계산한 값들을 이용해 나이브 베이즈 확률 P(Loan=1|CC=1, Online=1)을 계산하시오

9.88%

In [ ]:
# Using values calculated earlier
# P(CC=1 | Loan=1), P(Online=1 | Loan=1), P(Loan=1), P(CC=1 | Loan=0), P(Online=1 | Loan=0), and P(Loan=0)
p_cc_given_loan = 0.29791666666666666
p_online_given_loan = 0.60625
p_loan_accepted = 0.096
p_cc_given_not_loan = 0.29358407079646015
p_online_given_not_loan = 0.5957964601769912
p_loan_not_accepted = 0.904

# Calculating P(CC=1, Online=1)
p_cc_online = (p_cc_given_loan * p_online_given_loan * p_loan_accepted) + \
              (p_cc_given_not_loan * p_online_given_not_loan * p_loan_not_accepted)

# Calculating P(Loan=1 | CC=1, Online=1) using Naive Bayes
p_loan_given_cc_online = (p_cc_given_loan * p_online_given_loan * p_loan_accepted) / p_cc_online

p_loan_given_cc_online


0.09881705602499377

f. 이 값을 b의 피벗 테이블과 얻은 값과 비교하시오. 어느 것이 더 정확한 추정인가?

f가 더 정확하다

g. 이 표의 어떤 항목들이 $P(Loan=1|CC=1, Online=1)$을 계산하는 데 필요한가? 파이썬에서 이 데이터로 나이브 베이즈를 실행하시오. 학습 데이터셋에 대해 모델이 출력한 결과를 검토하고, $P(Loan=1|CC=1, Online=1)$ 에 해당하는 항목을 찾으시오. 이것을 e에서 얻은 값과 비교하시오

e에서 얻은 값: 9.88%, g에서 얻은 값: 9.90%

In [ ]:
from sklearn.naive_bayes import BernoulliNB

# Selecting features and target
X = bank_data[['CreditCard', 'Online']]
y = bank_data['Personal Loan']

# Train a Naive Bayes model
nb_model = BernoulliNB()
nb_model.fit(X, y)

# Predict probability for the case where CC=1 and Online=1
predicted_prob = nb_model.predict_proba([[1, 1]])[0][1]  # Probability of Loan=1 given CC=1, Online=1

predicted_prob


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but BernoulliNB was fitted with feature names
  warnings.warn(


0.09898133497742514